<a href="https://colab.research.google.com/github/Khangbeo17/documentation/blob/master/vicuna_13b_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, let's install the FastChat (Vicuna) repository.

In [1]:
!git clone https://github.com/thisserand/FastChat.git
%cd FastChat
!pip install -e .

Cloning into 'FastChat'...
remote: Enumerating objects: 816, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 816 (delta 71), reused 71 (delta 71), pack-reused 739
Receiving objects: 100% (816/816), 27.88 MiB | 11.33 MiB/s, done.
Resolving deltas: 100% (493/493), done.
/content/FastChat
Obtaining file:///content/FastChat
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git (to revision c612628045822f909020f7eb6784c79700813eda) to /tmp/pip-install-35ci7c1q/transformers_960bd65b4dc4479d9a0e1def18b19ff1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-35ci7c1q/transformers_960bd65b4dc4479d9a0e1def18b19ff1

Second, we will install the GPTQ-for-LLaMA repository which allows us to use GPTQ quantised models.

In [2]:
!mkdir repositories
%cd repositories
!git clone https://github.com/oobabooga/GPTQ-for-LLaMa.git -b cuda
%cd GPTQ-for-LLaMa
!python setup_cuda.py install
%cd ../..

/content/FastChat/repositories
Cloning into 'GPTQ-for-LLaMa'...
remote: Enumerating objects: 818, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 818 (delta 107), reused 100 (delta 100), pack-reused 698
Receiving objects: 100% (818/818), 469.29 KiB | 7.00 MiB/s, done.
Resolving deltas: 100% (497/497), done.
/content/FastChat/repositories/GPTQ-for-LLaMa
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

Okay, now we will download the quantised model weights.

In [3]:
!python download-model.py anon8231489123/vicuna-13b-GPTQ-4bit-128g

Traceback (most recent call last):
  File "/content/FastChat/download-model.py", line 196, in <module>
    links, sha256, is_lora = get_download_links_from_huggingface(model, branch)
  File "/content/FastChat/download-model.py", line 130, in get_download_links_from_huggingface
    fname = dict[i]['path']
KeyError: 0


To be able to run the web UI inside Colab we need virtual terminals. For this, we will use the screen library.

In [4]:
!sudo apt-get install screen

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  byobu | screenie | iselect ncurses-term
The following NEW packages will be installed:
  screen
0 upgraded, 1 newly installed, 0 to remove and 17 not upgraded.
Need to get 672 kB of archives.
After this operation, 1,029 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 screen amd64 4.9.0-1 [672 kB]
Fetched 672 kB in 1s (1,011 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package screen.
(Re

Launch the controler service in the background.

In [5]:
!screen -S controller -dm python -m fastchat.serve.controller --host "127.0.0.1"

Launch the model worker in the background.

In [6]:
!screen -S model_worker -dm python -m fastchat.serve.model_worker --model-path anon8231489123/vicuna-13b-GPTQ-4bit-128g --model-name vicuna-gptq --wbits 4 --groupsize 128 --host "127.0.0.1" --worker-address "http://127.0.0.1:21002" --controller-address "http://127.0.0.1:21001"

Wait a little bit (~30 seconds) before you run this cell to make sure that the model worker is up and running. Using the public URL you can interact with the Vicuna model.

In [3]:
!python -m fastchat.serve.gradio_web_server --controller-url "http://127.0.0.1:21001" --share

/usr/bin/python3: Error while finding module specification for 'fastchat.serve.gradio_web_server' (ModuleNotFoundError: No module named 'fastchat')
